In [2]:
#%%
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

import pandas as pd
import torch
import matplotlib.pyplot as plt
import torch.nn as nn
from transformers import AutoModelForCausalLM, AutoTokenizer
from huggingface_hub import snapshot_download
from tqdm import tqdm
import seaborn as sns

In [3]:
def get_reward_model():

    class GPTRewardModel(nn.Module):
        def __init__(self, model_path):
            super().__init__()
            model = AutoModelForCausalLM.from_pretrained(model_path, cache_dir='/scratch/henrypapadatos', device_map="auto")
            self.config = model.config
            self.config.n_embd = self.config.hidden_size if hasattr(self.config, "hidden_size") else self.config.n_embd
            self.config.output_hidden_states=True
            self.model = model
            self.transformer = model.model
            self.v_head = nn.Linear(self.config.n_embd, 1, bias=False).to(self.model.device)
            self.tokenizer = AutoTokenizer.from_pretrained(model_path)
            self.tokenizer.pad_token = self.tokenizer.unk_token
            self.PAD_ID = self.tokenizer(self.tokenizer.pad_token)["input_ids"][0]

        def get_device(self):
            return self.model.device

        def forward(
            self,
            input_ids=None,
            past_key_values=None,
            attention_mask=None,
            position_ids=None,
        ):
            """
            input_ids, attention_mask: torch.Size([bs, seq_len])
            return: scores: List[bs]
            """
            bs = input_ids.shape[0]
            transformer_outputs = self.transformer(
                input_ids,
                past_key_values=past_key_values,
                attention_mask=attention_mask,
                position_ids=position_ids,
            )
            hidden_states = transformer_outputs[0]
            scores = []
            rewards = self.v_head(hidden_states).squeeze(-1)
            for i in range(bs):
                c_inds = (input_ids[i] == self.PAD_ID).nonzero()
                c_ind = c_inds[0].item() if len(c_inds) > 0 else input_ids.shape[1]
                scores.append(rewards[i, c_ind - 1])
            return scores
        
    ## Load the model and tokenizer
    reward_model = GPTRewardModel("meta-llama/Llama-2-7b-chat-hf")
    reward_tokenizer = reward_model.tokenizer
    reward_tokenizer.truncation_side = "left"

    directory = snapshot_download("berkeley-nest/Starling-RM-7B-alpha", cache_dir='/scratch/henrypapadatos')
    for fpath in os.listdir(directory):
        if fpath.endswith(".pt") or fpath.endswith("model.bin"):
            checkpoint = os.path.join(directory, fpath)
            break

    # Load the model on the GPU  
    # reward_model.load_state_dict(torch.load(checkpoint, map_location=reward_device), strict=False)
    reward_model.load_state_dict(torch.load(checkpoint), strict=False)
    reward_model.eval().requires_grad_(False)

    return reward_model, reward_tokenizer

reward_model, reward_tokenizer = get_reward_model()

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Fetching 15 files:   0%|          | 0/15 [00:00<?, ?it/s]

In [12]:
def make_prompt(question, answer):
    prompt = question + answer
    prompt = prompt.replace('GPT4 Correct User:', '[INST] ')
    prompt = prompt.replace('GPT4 Correct Assistant:', '\n[/INST]')
    prompt = prompt.replace('<|end_of_turn|>', '')

    return prompt 

def get_activation(prompt, reward_model, reward_tokenizer):

    reward_device = reward_model.get_device()
    
    with torch.no_grad():
        #tokenize the input text
        encodings_dict = reward_tokenizer(
            prompt,
            truncation=True,
            max_length=2048,
            padding="max_length",
            return_tensors="pt",
        ).to(reward_device)

        tokenized_prompt = reward_tokenizer.convert_ids_to_tokens(encodings_dict['input_ids'][0])

        #We will keep the tokens corresponding to the answer given by the assistant only
        #to isolate it: start is the second occurence of the 'INST' token +2 (because we want to start reading after the ']' token.)
        #end is the first occurence of the '<unk>' token
        inst_token_index = [i for i, x in enumerate(tokenized_prompt) if x == 'INST'][1]
        start_token = inst_token_index + 2
        end_token = tokenized_prompt.index('<unk>')

        # print(tokenized_prompt)

        # print("partial prompt:")

        # print(tokenized_prompt[start_token:end_token])

        # raise ValueError('stop here')

        #get the model outputs
        output = reward_model.transformer(input_ids=encodings_dict['input_ids'], attention_mask=encodings_dict['attention_mask'])

        #taken the activations of the right layer and the right tokens
        activations = tuple([activation[0,start_token:end_token,:].cpu() for activation in output.hidden_states])

        #return the activations
        return activations

In [8]:
file = '../../results/Bo32_openchat_openchat_3.5_02_16_2024_00_08_55_feedback_poem_300-800_Starlingnew_comparison_gpt402_19_2024_18_09_07.csv'
df = pd.read_csv(file)

if 'Bo' in file:
    df = df[['type', 'prompt', 'BoN_1', 'BoN_1_preferred_1', 'BoN_1_preferred_1']]
    #change the column names
    df.columns = ['type', 'prompt', 'response', 'preferred_1', 'preferred_2']

#create a new column 'sycophancy_score' in the dataframe
df['activations_sycophantic'] = None
df['activations_not_sycophantic'] = None

#drop when type is 'wrote' or'not_wrote'
df = df[df['type'] != 'wrote']
df = df[df['type'] != 'not_wrote']
#reset index
df = df.reset_index(drop=True)
print(df.shape)

# df.head(10)

#drop elem when differant then NaN and when preferred_1 is different then preferred_2
for idx in df.index:
    if idx%3 != 0:
        continue
    #if gpt4 is not consistent, then the feedbacks are not considered to be strongly more or less positive
    if df.loc[idx+1, 'preferred_1'] != df.loc[idx+1, 'preferred_2'] or df.loc[idx+2, 'preferred_1'] != df.loc[idx+2, 'preferred_2']:
        df.drop([idx, idx+1, idx+2], inplace=True)
    #for our test, we want the feedbacks for prefix 'like' to be more positive and the feedbacks for prefix 'dislike' to be less positive
    elif df.loc[idx+1, 'preferred_1'] != 1 or df.loc[idx+2, 'preferred_1'] != 0:
         df.drop([idx, idx+1, idx+2], inplace=True)

#reset index
df = df.reset_index(drop=True)
print(df.shape)
# df.head(10)

new_df = pd.DataFrame()
for idx in df.index:
    if idx%3 != 0:
        continue
    row = {'type': 'like', 'prompt': df.loc[idx+1, 'prompt'], 'sycophantic_response': df.loc[idx+1, 'response'], 'not_sycophantic_response': df.loc[idx, 'response']}
    new_df = pd.concat([new_df, pd.DataFrame(row, index=[len(new_df)])])
    row = {'type': 'dislike', 'prompt': df.loc[idx+2, 'prompt'], 'sycophantic_response': df.loc[idx+2, 'response'], 'not_sycophantic_response': df.loc[idx, 'response']}
    new_df = pd.concat([new_df, pd.DataFrame(row, index=[len(new_df)])])

print(new_df.shape)
new_df.head(2)

(300, 7)
(183, 7)
(122, 4)


,type,prompt,sycophantic_response,not_sycophantic_response
0,like,GPT4 Correct User:Please comment briefly on th...,\n\nThe poem is a powerful exploration of the ...,\n\nThe poem vividly describes the concept of ...
1,dislike,GPT4 Correct User:Please comment briefly on th...,\nThe poem explores the concept of justice and...,\n\nThe poem vividly describes the concept of ...


In [ ]:
#iterate over the dataframe and get the activations for each row
new_df['activations_sycophantic'] = None
new_df['activations_not_sycophantic'] = None

for index, row in tqdm(new_df.iterrows(), total=new_df.shape[0]):
    question = row['prompt']
    
    answer = row['sycophantic_response']
    prompt = make_prompt(question, answer)
    activations = get_activation(prompt, reward_model, reward_tokenizer)
    new_df.at[index, 'activations_sycophantic'] = activations

    answer = row['not_sycophantic_response']
    prompt = make_prompt(question, answer)
    activations = get_activation(prompt, reward_model, reward_tokenizer)
    new_df.at[index, 'activations_not_sycophantic'] = activations

#save the dataframe to pkl format
new_df.to_pickle('../../datasets/open_ended_test_feedback_activations.pkl')

# df.head()

In [11]:
# print(df.iloc[0])
print(new_df.iloc[0]['activations_sycophantic'][0].shape)
print(new_df.iloc[0]['activations_not_sycophantic'][0].shape)
print(len(new_df.iloc[0]['activations_sycophantic']))

print(new_df.shape)
#remove the rows with missing activations
new_df = new_df[new_df['activations_sycophantic'].notna()]

print(new_df.shape)
print(type(new_df.iloc[0]['activations_sycophantic']))
print((new_df.iloc[0]['activations_sycophantic'][0]))



torch.Size([241, 4096])
torch.Size([314, 4096])
33
(122, 6)
(122, 6)
<class 'tuple'>
tensor([[ 0.0162, -0.0204, -0.0226,  ...,  0.0168,  0.0095,  0.0062],
        [-0.0129, -0.0027,  0.0078,  ...,  0.0015, -0.0012, -0.0013],
        [-0.0133, -0.0138,  0.0110,  ...,  0.0074, -0.0286,  0.0049],
        ...,
        [-0.0043,  0.0088, -0.0076,  ...,  0.0003, -0.0193, -0.0069],
        [-0.0032,  0.0199,  0.0079,  ...,  0.0173,  0.0023,  0.0045],
        [-0.0011,  0.0012, -0.0062,  ...,  0.0020, -0.0004, -0.0048]])
